In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zeotape/Products.csv
/kaggle/input/zeotape/Customers.csv
/kaggle/input/zeotape/Transactions.csv


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [3]:
customers=pd.read_csv("/kaggle/input/zeotape/Customers.csv")
transactions=pd.read_csv("/kaggle/input/zeotape/Transactions.csv")
products=pd.read_csv("/kaggle/input/zeotape/Products.csv")
transactions=transactions.merge(products.drop(columns="Price"),on="ProductID")
transactions["CompanyName"]=transactions["ProductName"].str.split().str[0]
transactions["ProductType"]=transactions["ProductName"].str.split().str[1:].str.join(" ")
transactions=pd.get_dummies(transactions,columns=["Category","CompanyName","ProductType"])
transactions

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,ProductName,Category_Books,Category_Clothing,...,ProductType_Novel,ProductType_Rug,ProductType_Running Shoes,ProductType_Smartphone,ProductType_Smartwatch,ProductType_Sweater,ProductType_T-Shirt,ProductType_Textbook,ProductType_Vase,ProductType_Wall Art
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,False,False,...,False,False,False,False,False,False,False,False,False,False
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,False,False,...,False,False,False,False,False,False,False,False,False,False
2,T00166,C0127,P067,2024-04-25 7:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,False,False,...,False,False,False,False,False,False,False,False,False,False
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,False,False,...,False,False,False,False,False,False,False,False,False,False
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 8:30:27,1,459.86,459.86,SoundWave Smartwatch,False,False,...,False,False,False,False,True,False,False,False,False,False
996,T00759,C0059,P037,2024-06-04 2:15:24,3,1379.58,459.86,SoundWave Smartwatch,False,False,...,False,False,False,False,True,False,False,False,False,False
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,SoundWave Smartwatch,False,False,...,False,False,False,False,True,False,False,False,False,False
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,SoundWave Smartwatch,False,False,...,False,False,False,False,True,False,False,False,False,False


In [4]:
categories=transactions.columns[8:]
categories

Index(['Category_Books', 'Category_Clothing', 'Category_Electronics',
       'Category_Home Decor', 'CompanyName_ActiveWear',
       'CompanyName_BookWorld', 'CompanyName_ComfortLiving',
       'CompanyName_HomeSense', 'CompanyName_SoundWave', 'CompanyName_TechPro',
       'ProductType_Biography', 'ProductType_Bluetooth Speaker',
       'ProductType_Cookbook', 'ProductType_Cookware Set',
       'ProductType_Desk Lamp', 'ProductType_Headphones', 'ProductType_Jacket',
       'ProductType_Jeans', 'ProductType_Laptop', 'ProductType_Mystery Book',
       'ProductType_Novel', 'ProductType_Rug', 'ProductType_Running Shoes',
       'ProductType_Smartphone', 'ProductType_Smartwatch',
       'ProductType_Sweater', 'ProductType_T-Shirt', 'ProductType_Textbook',
       'ProductType_Vase', 'ProductType_Wall Art'],
      dtype='object')

In [5]:
agg_list = [
    ("total_spent", "TotalValue", "sum"),
    ("avg_spent", "TotalValue", "mean"),
    ("total_quantity", "Quantity", "sum"),
]

for category in categories:
    agg_list.append((category, category, "sum"))

customer_spending=transactions.groupby("CustomerID").agg(
    **{col[0]: (col[1], col[2]) for col in agg_list}
).reset_index()


In [6]:
customer_features=customers.merge(customer_spending,on="CustomerID",how="left").fillna(0)
customer_features=pd.get_dummies(customer_features,columns=["Region"],drop_first=True)

In [7]:
customer_ids = customer_features["CustomerID"] 
customer_features = customer_features.drop(columns=["CustomerID", "CustomerName", "SignupDate"])
customer_features.head()

,total_spent,avg_spent,total_quantity,Category_Books,Category_Clothing,Category_Electronics,Category_Home Decor,CompanyName_ActiveWear,CompanyName_BookWorld,CompanyName_ComfortLiving,...,ProductType_Smartphone,ProductType_Smartwatch,ProductType_Sweater,ProductType_T-Shirt,ProductType_Textbook,ProductType_Vase,ProductType_Wall Art,Region_Europe,Region_North America,Region_South America
0,3354.52,670.904,12.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,False,False,True
1,1862.74,465.685,10.0,0.0,2.0,0.0,2.0,0.0,2.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,False,False,False
2,2725.38,681.345,14.0,0.0,1.0,1.0,2.0,4.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,False,False,True
3,5354.88,669.360,23.0,3.0,0.0,2.0,3.0,2.0,2.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,False,False,True
4,2034.24,678.080,7.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,False,False,False


In [8]:
scaler=StandardScaler()
scaled_features=scaler.fit_transform(customer_features)
knn=NearestNeighbors(n_neighbors=4)
knn.fit(scaled_features)

NearestNeighbors(n_neighbors=4)

In [9]:
top_look_alikes=dict()
for i,cust_id in enumerate(customer_ids[:20]):
    distances,ind=knn.kneighbors([scaled_features[i]])
    look_alike_cust=[]
    for k,j in enumerate(ind[0][1:4]):
        sim_score=1/(1+distances[0][k])
        look_alike_cust.append((customer_ids[j], sim_score))
    top_look_alikes[cust_id] = look_alike_cust

In [10]:
df=pd.DataFrame({"Map":[top_look_alikes]})
df.to_csv("Tanuj_Taneja_Lookalike.csv",index=False)
df.head()

,Map
0,"{'C0001': [('C0197', 0.9999999157063102), ('C0..."


In [11]:
lookalike_df = pd.DataFrame([
    {"CustomerID":cust_id,"Lookalikes":str(look_alikes)}
    for cust_id,look_alikes in top_look_alikes.items()
])
lookalike_df.to_csv("Tanuj_Taneja_Lookalike1.csv", index=False)
lookalike_df.head()
# was not sure about the output format so gave both as ouputs

,CustomerID,Lookalikes
0,C0001,"[('C0197', 0.9999999157063102), ('C0129', 0.17..."
1,C0002,"[('C0133', 1.0), ('C0164', 0.22474236336659878..."
2,C0003,"[('C0085', 1.0), ('C0095', 0.18269823882221084..."
3,C0004,"[('C0118', 1.0), ('C0181', 0.14572696926698941..."
4,C0005,"[('C0128', 1.0), ('C0007', 0.19528252943993127..."
